In [13]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import pickle

In [14]:
#prediction data
input_data = {
    'CustomerId':15984101,
    'Surname':'Watson',
    'CreditScore':600,
    'Geography':'France',
    'Gender': 'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [15]:
#Removing unecessary data
del input_data['CustomerId']
del input_data['Surname']
print(input_data)

{'CreditScore': 600, 'Geography': 'France', 'Gender': 'Male', 'Age': 40, 'Tenure': 3, 'Balance': 60000, 'NumOfProducts': 2, 'HasCrCard': 1, 'IsActiveMember': 1, 'EstimatedSalary': 50000}


In [16]:
#loading ML model
model = load_model('model.h5')

#loading encoders
with open('gender_encoder_file.pkl','rb') as file:
    gender_encoder = pickle.load(file)

with open('geo_encoder_file.pkl','rb') as file:
    geo_encoder = pickle.load(file)

#loading input scaler
with open('input_scaler_file.pkl','rb') as file:
    input_scaler = pickle.load(file)


In [17]:
#encoding prediction data and concatenating with the original prediction data
geo_data = geo_encoder.transform([[input_data['Geography']]]) #2d array expected
geo_data_df = pd.DataFrame(geo_data,columns=geo_encoder.get_feature_names_out())
input_data_df = pd.DataFrame([input_data]) #dictionary is treated as a scalar value- since the datafrme does not accept values without passing the index- we pass a list of dictionary so that it will be converted into a dataframe without index being provided.
input_data_df = pd.concat([input_data_df.drop(columns='Geography'),geo_data_df],axis=1)
input_data_df['Gender'] = gender_encoder.transform(input_data_df['Gender'])

input_data_df

c:\ANN_implementation\venv_new\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [18]:
#scaling prediction input
scaled_input = input_scaler.transform(input_data_df)


In [19]:
#predict if the employee will exit or not
predicted_output = model.predict(scaled_input)
print(predicted_output)
predicted_probability = predicted_output[0][0]
if predicted_probability>0.5:
    print("Customer will exit")
else:
    print("Customer will not exit")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
[[0.01962095]]
Customer will not exit
